## 1. Import Libraries

Necessary libraries to execute the code

In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

Matplotlib is building the font cache; this may take a moment.


## 2. Data Import
The following code cell is used to download the dataset files from the Kaggle website. It only needs to be executed once.  
Additionally, you will need to generate a personal Kaggle API Token in your account settings and install kaggle (pip install kaggle). 
In my case, when generating the Token, it was not automatically saved locally, so I created the json file manually and saved it in the .Kaggle folder in users. It follows this structure: 
{
    "username": "your_username",
    "key": "copy the API TOKEN the first time is generated, or not visible anymore"
}

In [6]:
def download_kaggle_dataset(dataset, path="data"):
     if not os.path.exists(path):
         os.makedirs(path)

     api = KaggleApi()
     api.authenticate()
     api.dataset_download_files(dataset, path=path, unzip=True)
     print("Dataset downloaded and extracted.")

if __name__ == "__main__":
    download_kaggle_dataset("barun2104/telecom-churn")

Dataset URL: https://www.kaggle.com/datasets/barun2104/telecom-churn
Dataset downloaded and extracted.


In [8]:
df=pd.read_csv('data/telecom_churn.csv')
print(df)

      Churn  AccountWeeks  ContractRenewal  DataPlan  DataUsage  \
0         0           128                1         1       2.70   
1         0           107                1         1       3.70   
2         0           137                1         0       0.00   
3         0            84                0         0       0.00   
4         0            75                0         0       0.00   
...     ...           ...              ...       ...        ...   
3328      0           192                1         1       2.67   
3329      0            68                1         0       0.34   
3330      0            28                1         0       0.00   
3331      0           184                0         0       0.00   
3332      0            74                1         1       3.70   

      CustServCalls  DayMins  DayCalls  MonthlyCharge  OverageFee  RoamMins  
0                 1    265.1       110           89.0        9.87      10.0  
1                 1    161.6       123 

**ATTRIBUTES DESCRIPTION**
- Churn = 
- AccountWeeks =
- ContractRenewal = 
- DataPlan =
- DataUsage = 
- CustServCalls =
- DayMins =
- DayCalls =
- MonthlyCharge =
- OverageFee =
- RoamMins = 
